In [2]:
# !pip install chromadb sentence_transformers

In [ ]:
import chromadb
from sentence_transformers import SentenceTransformer
import torch
from collections import defaultdict
import math

# ---------------------------------------------------------
# 1. 설정 (GPU 확인 등)
# ---------------------------------------------------------
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# ---------------------------------------------------------
# 2. 모델 로드 (저장할 때 썼던 그 모델!)
# ---------------------------------------------------------
print("모델 로드 중...")
model = SentenceTransformer("dragonkue/BGE-m3-ko").to(device)

# ---------------------------------------------------------
# 3. ChromaDB 연결 (다운로드 받은 폴더 경로 지정)
# ---------------------------------------------------------
# path="./chroma_db" 는 압축 푼 폴더 이름과 같아야 합니다.
client = chromadb.PersistentClient(path="./chroma_db")

# 컬렉션 가져오기 (create가 아니라 get_collection 사용)
collection = client.get_collection(name="patent_claims")

print(f"✅ 데이터베이스 로드 완료! 총 데이터 수: {collection.count()}개")



c:\Users\tower\anaconda3\envs\llm_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cpu
모델 로드 중...
✅ 데이터베이스 로드 완료! 총 데이터 수: 589049개


### 특허 단위 점수 집계(Late Fusion Aggregated Scoring)
- 출원번호 하나당 모든 ‘매칭된 청구항’들의 유사도 점수를 통계적으로 합성해서 특허 단위 점수를 만듦
- 단점: 독립항이 핵심인데 종속항이 우연히 많이 매칭되면 점수가 잘못 올라갈 수 있음

In [3]:
import numpy as np

def many_claim_dis(results, TOP_K):
    # ----------------------------------------
    # 0. Chroma 결과 파싱
    # ----------------------------------------
    ids        = results["ids"][0]
    docs       = results["documents"][0]
    metas      = results["metadatas"][0]
    distances  = results["distances"][0]

    parsed = []
    for i in range(len(ids)):
        parsed.append({
            "id": ids[i],
            "document": docs[i],
            "metadata": metas[i],
            "distance": distances[i]
        })

    # ----------------------------------------
    # 1. 출원번호 기준 그룹화
    # ----------------------------------------
    grouped = defaultdict(list)
    for r in parsed:
        app_no = r["metadata"]["patent_id"]
        grouped[app_no].append(r)

    # ----------------------------------------
    # 2. 특허 단위 점수 계산
    #    방법: claim similarity들의 평균 + 대표 claim 보정
    # ----------------------------------------

    def similarity(d):
        return 1-d


    def compute_patent_score(claims):
        sims = [similarity(c["distance"]) for c in claims]   # 새로운 sim

        sims_sorted = sorted(sims, reverse=True)
        top3 = sims_sorted[:3]
        top3_avg = sum(top3) / len(top3)
        max_sim = sims_sorted[0]

        claim_count = len(claims)
        count_bonus = min(1.0, claim_count / 10.0)

        final_score =  top3_avg * 0.6 + max_sim * 0.3 + count_bonus * 0.1
    
        return final_score

    # ----------------------------------------
    # 3. 특허 단위 재랭킹
    # ----------------------------------------
    aggregated = []
    for app_no, claims in grouped.items():
        score = compute_patent_score(claims)

        # 대표 claim은 거리(distance)가 가장 낮은 claim 선택
        rep_claim = sorted(claims, key=lambda x: x["distance"])[0]
         # claims에서 id와 metadata를 제외하고 document와 distance만 저장
        
        filtered_claims = [
            {
                "id": c["id"],
                "document": c["document"],
                "distance": c["distance"]
            }
            for c in claims
        ]

        aggregated.append({
            "patent_id": app_no,
            "score": score,
            "top_claim": rep_claim["document"],
            "top_claim_no": rep_claim["metadata"]["claim_no"],
            "claims_found": len(claims),
            "claims": filtered_claims
        })

    # 점수 높은 순으로 재랭킹
    aggregated = sorted(aggregated, key=lambda x: x["score"], reverse=True)

    final_response = aggregated[:TOP_K]
    return final_response

### 평가 함수

In [4]:
def eval(results, ipc_list, TOP_K):
    final_response = many_claim_dis(results, TOP_K)

    # 비교
    response_patent_id = [f['patent_id'] for f in final_response]

    print(ipc_list & set(response_patent_id))

    print("Precision:", len(ipc_list & set(response_patent_id)) / TOP_K)
    print("Recall:", len(ipc_list & set(response_patent_id)) / len(ipc_list))

    return final_response

### query가 2개 이상일 때 re-ranking해서 200건만 저장하는 함수

In [ ]:
# 200개 초과 검색되었을 때 re-ranking (z 정규화 수정ver)
import numpy as np

def multi_query_rerank(
    collection,
    model,
    query_list,
    per_query_top_k=200,
    final_top_k=200
):
    #------------------------------------------
    # 1) Q개의 query 문장 embedding
    #------------------------------------------
    query_embs = model.encode(query_list).tolist()


    #------------------------------------------
    # 2-1) query_list가 한 개일 경우, 검색 후 바로 return
    #------------------------------------------
    if len(query_list) == 1:
        return collection.query(query_embeddings=query_embs, n_results=per_query_top_k)


    #------------------------------------------
    # 2-2) query별 검색
    #------------------------------------------
    candidates = []  # 전체 후보 저장
    for emb in query_embs:
        r = collection.query(
            query_embeddings=[emb],
            n_results=per_query_top_k
        )
    
        distances = np.array(r["distances"][0])
        mean = distances.mean()
        std = distances.std() + 1e-9

        z_scores = (distances - mean) / std

        ids = r["ids"][0]
        docs = r["documents"][0]
        distances = r["distances"][0]
        metas = r["metadatas"][0]

        # 후보를 통합 리스트에 추가
        for pid, doc, meta, z, dist in zip(ids, docs, metas, z_scores, distances):
            candidates.append({
                "id": pid,
                "document": doc,
                'metadatas':meta,
                'distance':dist,
                'z-score':z
            })

    #------------------------------------------
    # 3) z-score 기준 오름차순 정렬 후 상위 final_top_k만 선택
    #------------------------------------------
    top_candidates = sorted(candidates, key=lambda x: x["z-score"])[:final_top_k]


    #------------------------------------------
    # 4) collection.query() 형식으로 재구성
    #------------------------------------------
    final_ids = [c["id"] for c in top_candidates]
    final_docs = [c["document"] for c in top_candidates]
    final_distances = [c["distance"] for c in top_candidates]
    final_metas = [c['metadatas'] for c in top_candidates]

    final_results = {
        "ids": [final_ids],
        "documents": [final_docs],
        "distances": [final_distances],
        "metadatas": [final_metas]
    }

    return final_results

### 정답 데이터 만들고 비교 1
G05D1/243, G05D1/648
환경에서 자연적으로 발생하는 신호를 캡처하는 수단, 예. 주변 광학, 음향, 중력 또는 자기 신호
작업 영역이나 공간 내에서 작업 수행, 예. 청소

In [13]:
query = ["청소 로봇 제어, 외부 전자 장치 소리 데이터 수집"]

results = multi_query_rerank(
    collection=collection,
    model=model,
    query_list=query,
    per_query_top_k=200,
    final_top_k=200
)

ipc={"1020230193702", "1020240075882", "1020240025833", "1020240009746", "1020230183389"}

TOP_K = 30
final_results = eval(results, ipc, TOP_K)

{'1020230193702', '1020240025833', '1020240075882'}
Precision: 0.1
Recall: 0.6


In [ ]:
query = [ '소리 데이터와 위치 정보를 이용해', '지도 기반으로 장치 위치를 식별하는 기술']

results = multi_query_rerank(
    collection=collection,
    model=model,
    query_list=query,
    per_query_top_k=200,
    final_top_k=200
)

ipc={"1020230193702", "1020240075882", "1020240025833", "1020240009746", "1020230183389"}

TOP_K = 30
eval(results, ipc, TOP_K)

{'1020230193702', '1020230183389'}
Precision: 0.06666666666666667
Recall: 0.4


[{'patent_id': '1020237013866',
  'score': 0.6796000301837921,
  'top_claim': '웨어러블 장치를 이용하여 제어가능한 장치의 위치를 식별하는 방법으로서,웨어러블 장치의 이미지 센서로부터 시각적 데이터를 수신하는 단계; 객체 인식 모듈에 의해, 상기 시각적 데이터에 기초하여 식별 데이터를 생성하는 단계;상기 식별 데이터를 이용하여, 제1 3차원(3D) 지도 및 제2 3차원 지도를 포함하는 복수의 3차원 지도들을 저장하는 지도 데이터베이스로부터 제1 3차원 지도를 식별하는 단계 -상기 제1 3D 지도는 제1 제어가능한 장치와 연관되고, 상기 제2 3D 지도는 제2 제어가능한 장치와 연관됨-; 상기 제1 3차원 지도의 시각적 포지셔닝 데이터에 기초하여 물리적 공간에서 상기 제1 제어가능한 장치의 위치를 획득하는 단계; 그리고상기 제1 제어가능한 장치의 위치의 임계 거리 내의 위치에서 상기 웨어러블 장치의 디스플레이 상에 사용자 인터페이스(UI) 객체를 렌더링하는 단계를 포함하는 것을 특징으로 하는 웨어러블 장치를 이용하여 제어가능한 장치의 위치를 식별하는 방법.',
  'top_claim_no': 1,
  'claims_found': 16,
  'claims': [{'id': '1020237013866_claim1',
    'document': '웨어러블 장치를 이용하여 제어가능한 장치의 위치를 식별하는 방법으로서,웨어러블 장치의 이미지 센서로부터 시각적 데이터를 수신하는 단계; 객체 인식 모듈에 의해, 상기 시각적 데이터에 기초하여 식별 데이터를 생성하는 단계;상기 식별 데이터를 이용하여, 제1 3차원(3D) 지도 및 제2 3차원 지도를 포함하는 복수의 3차원 지도들을 저장하는 지도 데이터베이스로부터 제1 3차원 지도를 식별하는 단계 -상기 제1 3D 지도는 제1 제어가능한 장치와 연관되고, 상기 제2 3D 지도는 제2 제어가능한 장치와 연관됨-; 상기 제1 3차원 지도의 시각적 포지셔닝 데이터에 기초하

### 정답 데이터 만들고 비교 2
G05D 1/43

In [15]:
# 같은 ipcNumber을 가진 출원번호
ipc_g05d143 = {"1020240014135",
"1020230114322",
"1020250071099",
"1020257016601",
"1020240039664",
"1020240037602",
"1020240024079",
"1020230147397",
"1020257029654",
"1020240062392",
"1020257011052",
"1020220110142",
"1020220132771",
"1020230173300",
"1020240085547",
"1020250040520",
"1020240013189",
"1020247037212",
"1020257009077",
"1020247018578",
"1020247018568",
"1020247018428",
"1020240079243",
"1020237031214",
"1020240026208",
"1020230177082",
"1020220120962",
"1020240003758",
"1020230134552",
"1020230127447",
"1020230127436"}

In [ ]:
query = [
    "주행 로봇의 위치·방향을 판단하고 제어하는 알고리즘",
    "지도 기반 위치 결정 및 경로 추종 제어 로직"
]

results = multi_query_rerank(
    collection=collection,
    model=model,
    query_list=query,
    per_query_top_k=200,
    final_top_k=200
)

TOP_K =30
final_results = eval(results, ipc_g05d143, TOP_K)

{'1020240003758', '1020247037212', '1020220120962'}
Precision: 0.1
Recall: 0.0967741935483871


### 정답 데이터 만들고 비교 3

G06Q50/10, G06K19/07

In [64]:
ipc_g06 = {
"1020220016944",
"1020240059583",
"1020240000411",
"1020220046632",
"1020230135386",
"1020230129173",
"1020230123448",
"1020220154077",
"1020230066241",
"1020240046742"
}

In [ ]:
query = [
    "사용자 단말 및 서버 간의 연동을 기반",
    "매핑 정보, DB 기반 서비스 실행",
    "식별 정보의 자동 수집, 전송 및 처리 과정을 통합"
]

results = multi_query_rerank(
    collection=collection,
    model=model,
    query_list=query,
    per_query_top_k=200,
    final_top_k=200
)

TOP_K = 30
f=eval(results, ipc_g06, TOP_K)

set()
Precision: 0.0
Recall: 0.0


### 정답 데이터 만들고 비교 4

In [69]:
ipc_g09g300_g09g33208 = {"1020230123509",
"1020230015809",
"1020230011205",
"1020230114342",
"1020220166481",
"1020220053782",
"1020210194515",
"1020240017940",
"1020210101348",
"1020240045365",
"1020240041098",
"1020240009599",
"1020240009333",
"1020240008829",
"1020220078865",
"1020230174203",
"1020220037942",
"1020230126254",
"1020230029941",
"1020230064399",
"1020240029529",
"1020230143723",
"1020230134748",
"1020230080308",
"1020230001366",
"1020230174009",
"1020240027415",
"1020220188338",
"1020220075085",
"1020210141296",
"1020240034098",
"1020237020882",
"1020240011182",
"1020230187763",
"1020230061180",
"1020247031712",
"1020230000119"}

In [ ]:
query = ['사용자 피로도 감지 기반 디스플레이 화면 조절', '휴먼 센서 통합 사용자 상태 분석', '디스플레이 자율 변형 조작']

results = multi_query_rerank(
    collection=collection,
    model=model,
    query_list=query,
    per_query_top_k=200,
    final_top_k=200
)

TOP_K = 30
final_results = eval(results, ipc_g09g300_g09g33208, TOP_K)

{'1020230187763'}
Precision: 0.03333333333333333
Recall: 0.02702702702702703
